# Multi-order Healpix

In [1]:
# imports
import numpy as np

import healpy 

from astropy import units
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

import astropy_healpix

from mocpy import MOC

# Set nside

In [2]:
nside = astropy_healpix.pixel_resolution_to_nside(0.1*units.arcsec)
nside

2097152

In [3]:
max_order = int(np.log2(nside))
max_order

21

## This seems one number to low..

# Find pixel indices inside the localization

    https://zonca.dev/2020/10/example-healpy-query_disc.html

## FRB localization centroid (fake)

In [4]:
frb_cen = SkyCoord(ra=326.1052292, dec=-40.9000278, unit='deg')

In [5]:
frb_cen.galactic.b

<Latitude -49.41466408 deg>

In [15]:
Angle(frb_cen.galactic.l), Angle(frb_cen.galactic.b)

(<Angle 0.74244708 deg>, <Angle -49.41466408 deg>)

In [16]:
moc = MOC.from_elliptical_cone(lon=Angle(frb_cen.galactic.l), 
                               lat=Angle(frb_cen.galactic.b),
                    a=Angle(0.4, units.arcsec), b=Angle(0.3, units.arcsec), 
                               pa=Angle(45, units.deg),
                               max_depth=max_order+1) # Note the +1

In [22]:
uniq = moc._uniq_format()
uniq

array([ 13890659872175,  13890659872184,  13890659872185,  13890659872186,
        13890659872517,  13890659872528,  55562639488693,  55562639488695,
        55562639488713,  55562639488714,  55562639488715,  55562639488716,
        55562639488718,  55562639488719,  55562639488748,  55562639488749,
        55562639488750,  55562639490077,  55562639490116,  55562639490120,
       222250557954685, 222250557954686, 222250557954687, 222250557954771,
       222250557954777, 222250557954778, 222250557954779, 222250557954797,
       222250557954799, 222250557954831, 222250557954842, 222250557954843,
       222250557954849, 222250557954850, 222250557954851, 222250557954868,
       222250557954870, 222250557954871, 222250557955004, 222250557955006,
       222250557960261, 222250557960263, 222250557960269, 222250557960304,
       222250557960305, 222250557960307, 222250557960472, 222250557960484],
      dtype=uint64)

In [23]:
type(uniq)

numpy.ndarray

In [24]:
moc.spatial_resolution_to_order(Angle(0.1, units.arcsec))

22

# Grab RA, DEC

In [25]:
level, ipix = astropy_healpix.uniq_to_level_ipix(uniq)
level, ipix

(array([20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22]),
 array([  9492613361071,   9492613361080,   9492613361081,   9492613361082,
          9492613361413,   9492613361424,  37970453444277,  37970453444279,
         37970453444297,  37970453444298,  37970453444299,  37970453444300,
         37970453444302,  37970453444303,  37970453444332,  37970453444333,
         37970453444334,  37970453445661,  37970453445700,  37970453445704,
        151881813777021, 151881813777022, 151881813777023, 151881813777107,
        151881813777113, 151881813777114, 151881813777115, 151881813777133,
        151881813777135, 151881813777167, 151881813777178, 151881813777179,
        151881813777185, 151881813777186, 151881813777187, 151881813777204,
        151881813777206, 151881813777207, 151881813777340, 151881813777342,
        15188181378259

## New nside

In [26]:
new_nside = astropy_healpix.level_to_nside(level)
new_nside

array([1048576, 1048576, 1048576, 1048576, 1048576, 1048576, 2097152,
       2097152, 2097152, 2097152, 2097152, 2097152, 2097152, 2097152,
       2097152, 2097152, 2097152, 2097152, 2097152, 2097152, 4194304,
       4194304, 4194304, 4194304, 4194304, 4194304, 4194304, 4194304,
       4194304, 4194304, 4194304, 4194304, 4194304, 4194304, 4194304,
       4194304, 4194304, 4194304, 4194304, 4194304, 4194304, 4194304,
       4194304, 4194304, 4194304, 4194304, 4194304, 4194304])

## lon, lat

In [27]:
lon, lat = astropy_healpix.healpix_to_lonlat(ipix, new_nside, order='nested')
lon.to('deg'), lat.to('deg')

(<Longitude [0.74234918, 0.74245022, 0.74255042, 0.74244938, 0.74234835,
             0.74244855, 0.74237527, 0.74237486, 0.74247631, 0.74242579,
             0.74247589, 0.74252641, 0.74252599, 0.74257609, 0.74252474,
             0.74257484, 0.74252432, 0.74237277, 0.74252391, 0.74242287,
             0.74238832, 0.74236306, 0.74238811, 0.74233759, 0.74233739,
             0.74231213, 0.74233718, 0.74228645, 0.74228624, 0.74248915,
             0.7425142 , 0.74253925, 0.74243884, 0.74241358, 0.74243863,
             0.74256409, 0.74256388, 0.74258893, 0.742562  , 0.74256179,
             0.74228603, 0.74228583, 0.74228562, 0.74231025, 0.7423353 ,
             0.74233509, 0.74251107, 0.74246055] deg>,
 <Latitude [-49.4146961 , -49.4146961 , -49.41464853, -49.41464853,
            -49.41464853, -49.41460096, -49.41474367, -49.41471988,
            -49.41474367, -49.41474367, -49.41471988, -49.41471988,
            -49.4146961 , -49.41467231, -49.41462475, -49.41460096,
            -49.